# Life insurance

In [26]:
# Necessary libraries
library(readr)
library(ggplot2)
library(tidyverse)
library(plotly)
library(purrr)
library(DT)
library(data.table)

# The Human Mortality Database 

## Download data

We can download the data from https://www.mortality.org/Data/ZippedDataFiles.  After registering for a free account, we can download a txt file.


![](data.PNG)



## Data processing

### Use male data


In [27]:
file_list <- list.files(path="male")

In [28]:
#create a list of the files from your target directory


code=gsub(".mltper_1x1.txt","",file_list)

In [29]:
file_list_full <- list.files(path="male",full.names = T)

In [30]:


l=lapply(file_list_full, fread)
names(l)=code

f=rbindlist(l,use.names = T,fill=T,idcol="file")


Warning message in FUN(X[[i]], ...):
“Stopped early on line 47. Expected 2 fields but found 0. Consider fill=TRUE and comment.char=. First discarded non-empty line: <<## 	Products>>”


In [31]:



country_code=readxl::read_excel("country_code.xlsx")

final=f %>% left_join(country_code,by=c("file"="code"))
final  %>% head()

file,Year,Age,mx,qx,ax,lx,dx,Lx,Tx,ex,##,Insurance rationale,Country
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
AUS,1921,0,0.07653,0.07253,0.28,100000,7253,94762,5910278,59.1,NA,NA,Australia
AUS,1921,1,0.01434,0.01424,0.5,92747,1320,92087,5815516,62.7,NA,NA,Australia
AUS,1921,2,0.00605,0.00603,0.5,91427,551,91151,5723428,62.6,NA,NA,Australia
AUS,1921,3,0.0042,0.00419,0.5,90876,381,90686,5632277,61.98,NA,NA,Australia
AUS,1921,4,0.00325,0.00325,0.5,90495,294,90348,5541591,61.24,NA,NA,Australia
AUS,1921,5,0.00251,0.00251,0.5,90201,226,90088,5451243,60.43,NA,NA,Australia



### Exercise

Write a R code to processing data for female from https://www.mortality.org/Data/ZippedDataFiles.



In [ ]:
## Solution here



## Selecting a year

### Data for those born in 2002


We select data for those born in 2002.


In [32]:
data=final %>% select(Country,Year, Age, qx,lx,dx) %>% mutate(px=1-as.numeric(qx)) %>% filter(Year==2002,!is.na(Country))

data[Age=="110+",]$Age="110"

data =data %>%  mutate_at(names(data)[-1], as.numeric)
glimpse(data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `px = 1 - as.numeric(qx)`.
Caused by warning:
! NAs introduced by coercion”


Rows: 4,995
Columns: 7
$ Country <chr> "Australia", "Australia", "Australia", "Australia", "Australia…
$ Year    <dbl> 2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002, 20…
$ Age     <dbl> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …
$ qx      <dbl> 0.00529, 0.00047, 0.00034, 0.00031, 0.00015, 0.00014, 0.00011,…
$ lx      <dbl> 100000, 99471, 99424, 99391, 99360, 99345, 99332, 99320, 99303…
$ dx      <dbl> 529, 47, 34, 30, 15, 14, 11, 17, 10, 16, 19, 7, 11, 19, 25, 26…
$ px      <dbl> 0.99471, 0.99953, 0.99966, 0.99969, 0.99985, 0.99986, 0.99989,…


### Exercise

Write R code to filter data for those female born in 2004 and 2000.

In [ ]:
## Solution here



# Data description

+ Year: Year of observing data

+ Age: Age group 

+ q(x): Probability of death between ages x and x+1

+ p(x): Probability of alive between ages x and x+1


+ l(x): Number of survivors at exact age x, assuming l(0) = 100,000

+ d(x): Number of deaths between ages x and x+1







## One-year mortality rate 

### Indiviual probability


Given a male born in USA in 2002, what is the estimate probability of dying in year of age 40?


In [33]:
data %>% filter(Country=="U.S.A.",Age==40) %>% select(qx)


qx
<dbl>
0.00268


Which country has the highest mortality rate at age 40 for thos born in 2002?


In [34]:
data %>% filter(Age==40) %>% select(Country,qx)  %>% filter(qx==max(qx))

Country,qx
<chr>,<dbl>
Russia,0.01294


### Exercise

1. Given a female born in Australia in 2000, what is the estimate probability of dying in year of age 60?

2. Which country has the lowest mortality rate at age 60 for those female born in 2000?

In [ ]:
## Solution here





### One-year log mortality rate over ages

In [35]:
t=data %>% ggplot(aes(x=Age, y=log(qx),col=Country,group=1))+geom_line()
ggplotly(t)

HTML widgets cannot be represented in plain text (need html)

### Exercise

Plot One-year log mortality rate over ages for those female born in 2000.



In [ ]:
## Solution here


## Survival probabilities 

### Individual probabilities

Given a baby born in 2002, the probability he/she can live over a certain age is the cumulative product of probability of alive from the current age to that age.

For example, compute the survival probability over the first 5 years of a male baby born in 2002 in USA.


In [36]:
data %>% filter(Country=="U.S.A.",Age %in% c(0:5)) %>% summarize(sp=prod(px))


sp
<dbl>
0.9907133


Compute the survival probability over the next 50 years of a 20-year male born in 2002 in USA.


In [37]:
data %>% filter(Country=="U.S.A.",Age %in% c(20:70)) %>% summarize(sp=prod(px))

sp
<dbl>
0.6919628


### Exercise 

1. Compute the survival probability over the first 25 years of a female baby born in 2002 in Australia.

2. Compute the survival probability over the next 70 years of a 20-year female born in 2000 in Australia.



In [ ]:
# Solution here



### Survival probabilities over ages

Estimate survival probabilities of a male born in 2002 over ages


In [38]:
t=data %>%group_by(Country)%>% mutate(cpx=cumprod(px))%>% ggplot(aes(x=Age, y=cpx,col=Country,group=1))+geom_line()
ggplotly(t)


HTML widgets cannot be represented in plain text (need html)

Given a male born in 2002 and still alive in 2022, the probability he/her can live over a certain age is the cumulative product of probability of alive from age 20 to that age.


In [39]:
t=data %>% filter(Age>20) %>%group_by(Country)%>% mutate(cpx=cumprod(px))%>% ggplot(aes(x=Age, y=cpx,col=Country,group=1))+geom_line()
ggplotly(t)


HTML widgets cannot be represented in plain text (need html)


### Exercise 

1. Plot survival probabilities of a female born in 2002 over ages

2. Given a female born in 2002 and still alive in 2022, the probability he/her can live over a certain age is the cumulative product of probability of alive from age 20 to that age.


In [ ]:
# Solution here




# Life Expectancy

## Compute the life expectancy

- A male was born in 2002 in USA. Compute his life expectancy when he was 20 years old.

- Let $K$ be the age he dies. $K$ could receive values from 20 to 110 (as the survival probability at age 111 is 0).

+ The probability $K =20$ (he died in that year) is $q_{20}$, where $q_{20}$ is the probability of dying in age 20 for people was born in 2002.

+ The probability $K =21$ (he lived through age 20 and died in age 21) is $p_{20}q_{21}$, where $p_{20}$ is the probability of surving in age 20 for people was born in 2002.

+ The probability $K =22$ (she lived through ages 20, 21 and died in age 22) is $p_{20}p_{21}q_{22}$.

+ Similar arguments for other values of $K$. For instance, the probability $K=110$ is $p_{20}...p_{109}q_{110}$
    
Thus the expected value of $K$ is computed as:

$$E[K] =20*q_{20}+21*p_{20}q_{21}+22*p_{20}p_{21}q_{22}+...+110*p_{20}...p_{109}q_{110}$$

We can rewrite this as

$$E[K] =20*(1-p_{20})+21*p_{20}(1-p_{21})+22*p_{20}p_{21}(1-p_{22})+...+110*p_{20}...p_{109}(1-p_{110})$$

As a result, we can obtain:

$$E[K] =20+p_{20}+p_{20}p_{21}+p_{20}p_{21}p_{22}+...+p_{20}...p_{109}$$


The life expectancy is thus computed as the sum of cumulative survival probabilities

$$p_{20}+p_{20}p_{21}+p_{20}p_{21}p_{22}+...+p_{20}...p_{109}$$



In [40]:
data %>% filter(Country=="U.S.A.",Age>=20) %>% summarize(sum(cumprod(px)))

sum(cumprod(px))
<dbl>
54.91941



## Plotting life expectancies by Country


In [41]:
t= data %>% filter(Age>=20) %>% group_by(Country)%>% summarize(el=sum(cumprod(px))) %>% ggplot(aes(x=fct_reorder(Country, el),el))+geom_point()+coord_flip()+labs(x="Country", title="Life expectancy of those 20-year old male born in 2002")

ggplotly(t)


HTML widgets cannot be represented in plain text (need html)


## Exercise

Plot the life expectancy of those 25-year old female born in 2002 by country.


In [ ]:
# Solution here




# Life insurance products' evaluation 



## Stochastic cashflows's evaluation 


A 20-year-old US male (born in 2002) has been offered 2 options. Option 1: he will be surely received 10,000 EUR at the age 35. Option 2: he will receive 10,000 EUR if she is still alive at age 35. Assume the interest rate  is constant at 2%. The life table used for this customer is given below. How much should he pay now for each option? 


In [42]:
data[Country=="U.S.A."& Age>=20,] %>% datatable()

HTML widgets cannot be represented in plain text (need html)

Option 1:

In [43]:
# PV of guaranteed payment of 10,000 in 5 years
(PV <- 10000 * (1+0.02) ^ (-15))

[1] 7430.147

Option 2:

In [44]:
life_table=data[Country=="U.S.A."& Age>=20,]

# 5 year survival probabilities of (20)
kpx=prod(life_table[Age %in% (20:34)]$px )
# EPV of pure endowment of 10,000 in 5 years for (20)
PV*kpx


[1] 7271.773



## Exercise 

A 25-year-old Australian female (born in 2000) has been offered an option that he will receive 10,000 EUR if she is still alive at age 35. Assume the interest rate  is constant at 5%. How much should she pay now for the option? 


In [ ]:
# Solution here



## Simple life insurance 

### Example

Consider a very simple life insurance product: a product sold to a US male customer aged 20 (born in 2002) that pays 10,000 EUR at the end of the year of death if death occurs at a given age 30. Assume the interest rate is constant at 1%, what is the expected present value of this simple life insurance?



In [45]:
life_table=data[Country=="U.S.A."& Age>=20,]
life_table[Age %in% (20:30)] %>% datatable()


HTML widgets cannot be represented in plain text (need html)

In [46]:
# The probability for the customer survives from age to 20 to 29
(kpx <- prod(life_table[Age %in% (20:29)]$px ))
# The probability of the customer dies at exactly age 30
life_table[Age==30]$qx

kqx <-kpx * life_table[Age==30]$qx

# Discount factor
discount_factor <- (1 + 0.01) ^ - 11

# EPV of the simple life insurance
10000 * discount_factor * kqx


[1] 0.9861767

[1] 0.00144

[1] 12.72864

### Exercise

Consider a very simple life insurance product: a product sold to a Australian female customer aged 20 (born in 2002) that pays 10,000 EUR at the end of the year of death if death occurs at a given age 40. Assume the interest rate is constant at 5%, what is the expected present value of this simple life insurance?

In [ ]:
# Solution here




## Whole life insurance 

### Example 

Consider an US male aged 20 (born in 2002). He buys a whole life insurance that gives 10,000 EUR if he dies at any age in the future. Assume the interest rate is constant at 3%, compute the expected present value of this product.


### Solution


In [47]:

life_table %>% datatable()


HTML widgets cannot be represented in plain text (need html)

In [48]:

# The probability vector of dying at exactly age: 20 to 109
p_benefits= cumprod(c(1,life_table[Age %in% (20:109)]$px))* life_table[Age %in% (20:110)]$qx

# vector of benefits
benefits= c(rep(10000,110-20+1))
discount= (1+0.03)^-(1:91)
benefits_discount= benefits*discount
expected_PV= sum(p_benefits*benefits_discount)
expected_PV

[1] 2142.082

### Exercise 

Consider an Australian female aged 20 (born in 2000). She buys a whole life insurance that gives 10,000 EUR if she dies at any age in the future. Assume the interest rate is constant at 5%, compute the expected present value of this product.


In [ ]:
# Solution here





## Exercise


1. A man aged 20. He buys a temporary life insurance that gives 10,000 EUR if he dies at any age in the next 50 years. The life table used for this customer is given below. Assume the interest rate is constant at 3%, compute the expected present value of this product.


In [49]:

life_table %>% datatable()


HTML widgets cannot be represented in plain text (need html)

In [ ]:
# Solution here



2. Miss Cathleen, aged 48, seeks for financial protection in case she would die around her retirement. The life table used for this customer is given below. To protect her growing-up children she wants to insure a benefit of 40,000 EUR for death between age 55 and 75. Assume the interest rate is constant at 5%, compute the expected present value of this product.



In [ ]:
# Solution here

In [50]:
life_table %>% datatable()


HTML widgets cannot be represented in plain text (need html)




3. A man aged 20. He buys a deferred life insurance that gives 10,000 EUR if he dies at any age after the next 30 years. The life table used for this customer is given below. Assume the interest rate is constant at 3%, compute the expected present value of this product.


In [ ]:
# Solution here

In [51]:
life_table %>% datatable()

HTML widgets cannot be represented in plain text (need html)